## Εργασία 3 ##

Καλωσήρθατε στην τρίτη σας εργασία. Η εργασία αυτή έχει σκοπό να σας βοηθήσει να εμπεδώσετε τα σύνολα μοντέλων.

Στην εργασία αυτή θα πρέπει να συμπληρώσετε κώδικα Python 3 στα σημεία που αναφέρουν # YOUR CODE HERE. Μην τροποποιείτε τον κώδικα που βρίσκεται εκτός αυτών των περιοχών.

Πρωτού παραδόσετε την εργασία σας σιγουρευτείτε ότι ο κώδικας σε όλα τα κελιά τρέχει σωστά. Για το σκοπό αυτό επιλέξτε από το μενού **Χρόνος εκτέλεσης (runtime) -> Επανεκίνηση περιόδου λειτουργίας και εκτέλεση όλων**.

Συμπληρώστε το όνομα (**NAME**) και το **AEM** σας παρακάτω **εντός των διπλών εισαγωγικών**:

In [16]:
NAME = "Παπαδάκης Κωνσταντίνος Φώτιος"
AEM = "10371"

**1 (2 μονάδες)** Σε αυτήν την εργασία θα δουλέψουμε με ένα σύνολο δεδομένων δυαδικής ταξινόμησης, από το δημόσιο αποθετήριο δεδομένων και πειραμάτων μηχανικής μάθησης [openml.org](https://www.openml.org/). Μπορείτε να δείτε πως φορτώνουμε σύνολα δεδομένων από το openml.org μέσα από την sklearn μελετώντας την αντίστοιχη [τεκμηρίωση](https://scikit-learn.org/stable/datasets/loading_other_datasets.html#downloading-datasets-from-the-openml-org-repository). Συγκεκριμένα θέλουμε να δουλέψουμε με την έκδοση 1 του Pima Indians Diabetes Database, το οποίο αφορά τη διάγνωση διαβήτη. Χωρίστε το σύνολο δεδομένων σε δεδομένα εκπαίδευσης (X_train, y_train) και ελέγχου (X_test, y_test) σε ποσοστό 70% και 30% αντίστοιχα με τη συνάρτηση train_test_split (τιμή για random_state βάλτε 0).

In [17]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# Fetch the diabetes dataset from OpenML
diabetes = fetch_openml(name='diabetes', version=1)
X, y = diabetes.data, diabetes.target
# Split the dataset into training 70% and test 30% sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

In [18]:
"""Τεστ ορθής ανάγνωσης και διαχωρισμού του συνόλου δεδομένων"""
assert X_train['mass'][0] == 33.6
assert X_test['pedi'][34] == 0.512

**2 (4 μονάδες)** Υλοποιήστε μια ντετερμινιστική εκδοχή της μεθόδου των τυχαίων υποχώρων, η οποία χτίζει τόσα μοντέλα όσες και οι μεταβλητές εισόδου, κάθε ένα από τα οποία αγνοεί και μία διαφορετική μεταβλητή εισόδου. Π.χ. το πρώτο μοντέλο αγνοεί την πρώτη, το δεύτερο αγνοεί τη δεύτερη κτλ. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. 

In [19]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class RandomSubspaceDet:
    def __init__(self, estimator=DecisionTreeClassifier()):
        # Initialize the class attributes
        self.estimator = estimator
        self.models = []
        self.ignore_indices = []

    def fit(self, X_train, y_train):
        # Reset the models and ignore indices
        self.models = []
        self.ignore_indices = []

        # Get the number of features
        n_features = X_train.shape[1]

        # For each feature index i, remove the i-th column from X_train
        for i in range(n_features):
            # Remove the i-th feature from the training data
            X_sub = np.delete(X_train, i, axis=1)
            # Clone the estimator and fit it to the reduces data
            model = clone(self.estimator)
            model.fit(X_sub, y_train)
            # Store the fitted model and the index of the ignored feature
            self.models.append(model)
            self.ignore_indices.append(i)

    def predict(self, X):
        # Initialize an empty list to store predictions
        predictions = []

        # For each trained model
        for model, idx in zip(self.models, self.ignore_indices):
            # Remove the i-th feature from the test data
            X_sub = np.delete(X, idx, axis=1)
            # Calculate the reduced input
            preds = model.predict(X_sub)
            # Append the predictions to the list
            predictions.append(preds)

        # Convert the list of predictions into a 2D NumPy array
        predictions = np.array(predictions)

        # Majority vote across models (row-wise mode)
        final_preds = []
        for i in range(predictions.shape[1]):
            # Returns all unique predicted labels and their frequencies.
            values, counts = np.unique(predictions[:, i], return_counts=True)
            # Get the label with the highest frequency
            majority_vote = values[np.argmax(counts)]
            # Append this to the list of final predictions.
            final_preds.append(majority_vote)

        return np.array(final_preds)


In [20]:
"""Τεστ ορθής υλοποίησης RandomSubspaceDet"""
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

rs = RandomSubspaceDet(estimator=DecisionTreeClassifier(random_state=1))
rs.fit(X_train, y_train)
assert round(accuracy_score(rs.predict(X_test), y_test), 4) == 0.7446

**3 (4 μονάδες)** Υλοποιήστε τη μέθοδο AdaBoost όπως παρουσιάστηκε στο μάθημα. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. Χρησιμοποιήστε την παράμετρο sample_weight της fit του βασικού μοντέλου για να ορίσετε τα βάρη των παραδειγμάτων εκπαίδευσης. 

In [29]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class AdaBoost:
    def __init__(self, n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1)):
        self.n_estimators = n_estimators
        self.estimator = estimator
        self.models = []
        self.alphas = []
        self.forward_map = {}
        self.reverse_map = {}

    def fit(self, X_train, y_train):
        y_train = np.array(y_train)

        # Map the two classes to -1 and +1
        classes = np.unique(y_train)
        self.forward_map = {classes[0]: -1, classes[1]: 1}
        self.reverse_map = {-1: classes[0], 1: classes[1]}
        y_binary = np.vectorize(self.forward_map.get)(y_train)

        n_samples = X_train.shape[0]
        weights = np.ones(n_samples) / n_samples

        for _ in range(self.n_estimators):
            model = clone(self.estimator)
            model.fit(X_train, y_binary, sample_weight=weights)

            y_pred = model.predict(X_train)
            y_pred = np.sign(y_pred)  # ensure predictions are -1 or +1

            # Compute weighted error
            incorrect = (y_pred != y_binary)
            err = np.dot(weights, incorrect)
            err = max(err, 1e-10)

            alpha = 0.5 * np.log((1 - err) / err)

            # Update weights
            weights *= np.exp(-alpha * y_binary * y_pred)
            weights /= np.sum(weights)

            self.models.append(model)
            self.alphas.append(alpha)

    def predict(self, X):
        final_pred = np.zeros(X.shape[0])

        for model, alpha in zip(self.models, self.alphas):
            pred = model.predict(X)
            pred = np.sign(pred)  # ensure -1 or +1
            final_pred += alpha * pred

        y_pred_binary = np.sign(final_pred)
        y_pred_binary[y_pred_binary == 0] = 1  # resolve ties to class 1

        return np.vectorize(self.reverse_map.get)(y_pred_binary)


In [30]:
"""Τεστ ορθής υλοποίησης AdaBoost"""
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

ab = AdaBoost(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=2, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.7706 


In [31]:
#Ίδιο αποτέλεσμα παίρνουμε και με την κλάση της sklearn. Ωστόσο υπάρχουν γενικότερα διαφορές στην υλοποίηση της σε σ΄χέση με αυτήν που είδαμε στη θεωρία, όπως η υποστήριξη παραπάνω από 2 κλάσεων και ο διαφορετικός χειρισμός της τυχαιότητας στην εκπαίδευση των επί μέρους μοντέλων.
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ab = AdaBoostClassifier(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=2, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.7706